In [1]:
using Erdos
using Distributions
using Gadfly
using Clustering
using MatrixNetworks

g = readgraph(:as22july06);
a = adjacency_list(g::AGraph);

N = size(a)[1];
A0 = sparse(1:N,1:N,0*vec(ones(Int64,N,1)),N,N);

for i=1:N
    l = a[i];
    for j=1:length(l)
        A0[i,l[j]]=Int8(1);
        A0[l[j],i]=Int8(1);
    end
end
A,p = largest_component(A0);
A0=0;p=0;a=0;g=0;
gc()
dropzeros(A);

In [2]:
d = sum(A,2);
s = sum(d);

r = 100;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[55.1693, 39.8423, 37.4582, 35.6205, 32.8834, 32.956, 30.3229, 27.3132, 25.3254, 25.0209, 24.0735, 22.8298, 21.5613, 21.3384, 20.8339, 20.2921, 20.1245, 19.4307, 18.6252, 18.2628, 18.0632, 17.6689, 16.9593, 16.5094, 16.0631, 16.217, 15.9762, 15.604, 15.0362, 14.9429, 14.7064, 14.4576, 13.9687, 13.7519, 13.557, 13.3767, 13.1008, 13.2209, 13.1161, 12.8225, 12.6999, 12.6406, 12.3654, 12.4197, 12.2066, 12.1239, 12.0827, 12.0228, 11.8489, 11.7409, 11.533, 11.4918, 11.4608, 11.1698, 11.1057, 11.0673, 10.7383, 10.698, 10.5383, 10.3099, 10.3766, 10.1624, 9.85059, 10.0978, 9.99741, 9.57909, 9.72011, 9.6619, 9.61748, 9.60016, 9.37238, 9.39717, 9.36436, 9.19043, 9.30698, 9.26567, 9.141, 9.05344, 9.03023, 9.06982, 8.8222, 8.94182, 8.69234, 8.80052, 8.68951, 8.53645, 8.55995, 8.57828, 8.43673, 8.33116, 8.38796, 8.38054, 8.42085, 8.31977, 8.3714, 8.3172, 8.33979, 8.27157, 8.21502, 8.25929]

In [20]:
include("Algo/Partition.jl")
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")

Sig = v0[:,1:18]';

n_clusters = 200;
n_it_Kmeans = 1000;
n_updates = 100;


community = vec(zeros(N,1));
community0 = vec(zeros(N,1));
Q_best = 0;
n_c_best = 0;
n_c = 0;
# Probability mass for sampling the initial centroids
p = vec(d/s);
n_updates_best = 0;

for k =1:n_it_Kmeans

	community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
	n_c = length(unique(community0));

    	if Q>Q_best
		community = community0;
		Q_best = Q;
		n_c_best = n_c;
		n_updates_best = n_updates;

		
    	end
end	
  
	community = Reorder(community);

	print("Number of updates: ")
	print(n_updates_best)
	print("\n")
	print("Number of communities: ")
	print(n_c_best)
	print("\n")

	print("Modularity: ")
	print(Q_best)
	print("\n")

Number of updates: 1
Number of communities: 20
Modularity: 0.5092989402625958


In [3]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 500;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:100
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


Number of updates: 1
Number of communities: 48
Modularity: 0.5147485631501421
n: 52
